# 1. Introduction

Factor analysis for mixed data (FAMD) is a principal component method for exploring data with both continuous and categorical variables. It is roughly a combination of principal component analysis (PCA) for continuous variables and multiple correspondence analysis (MCA) for categorical variables. To learn more about FAMD, see an excellent [tutorial](http://www.sthda.com/english/articles/31-principal-component-methods-in-r-practical-guide/115-famd-factor-analysis-of-mixed-data-in-r-essentials/) using the `FactoMineR` package.

In this series, I will use three commonly used packages in R ([FactoMineR](https://cran.r-project.org/web/packages/FactoMineR/FactoMineR.pdf) and [PCAmixdata](https://cran.r-project.org/web/packages/PCAmixdata/PCAmixdata.pdf)) and Python ([prince](https://github.com/MaxHalford/prince))to performance FAMD on the Telco customer churn dataset, to gain insights into the relationships between various aspects of customer behaviour.

Note, I used the [SoS kernel](https://vatlab.github.io/sos-docs/) for this analysis, which allows both Python and R analysis in the same notebook without using the Jupyter R magic commands. I found SoS handled certain aspects of R performance better than the R magics, so I encourage those who are interested to check it out. :)

# 2. Import and pre-process data

Here, I will import the cleaned Telco dataset in both R and Python. 

As `Calculated_TotalCharges` is highly correlated with `tenure` and `MonthlyCharges`, it will be excluded from analysis.

Also, all three packages automatically normalize the numerical variables, so I will not do so before hand.

In [2]:
df <- read.csv('https://github.com/nd823/data-cleaning/raw/master/telco_cleaned_Jun13.csv')

df <- within(df, rm('Calculated_TotalCharges'))

In [3]:
import pandas as pd

df = pd.read_csv('https://github.com/nd823/data-cleaning/raw/master/telco_cleaned_Jun13.csv')

df.drop('Calculated_TotalCharges', axis=1, inplace=True)

# 3. Factor analysis of mixed data

## 3.1 `FactoMineR`

In [9]:
## Import libraries
library(FactoMineR)
library(factoextra)

## PCA
res.famd <- FAMD(df 
                 sup.var = 19, 
                 graph = FALSE, 
                 ncp=25)

## Inspect principal components
get_eigenvalue(res.famd)

Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Welcome! Related Books: `Practical Guide To Cluster Analysis in R` at https://goo.gl/13EFCZ


,eigenvalue,variance.percent,cumulative.variance.percent
Dim.1,4.5098513842,20.49932447,20.49932
Dim.2,2.8024012638,12.73818756,33.23751
Dim.3,1.8247464408,8.29430200,41.53181
Dim.4,1.1490819680,5.22309985,46.75491
Dim.5,1.0495900326,4.77086378,51.52578
Dim.6,1.0144020899,4.61091859,56.13670
Dim.7,0.9993340111,4.54242732,60.67912
Dim.8,0.9842737776,4.47397172,65.15310
Dim.9,0.9039827483,4.10901249,69.26211
Dim.10,0.8465510646,3.84795938,73.11007


## 3.2 `PCAmixdata`

In [10]:
library(PCAmixdata)

## Split quantitative and qualitative variables
split <- splitmix(df)

## PCA
res.pcamix <- PCAmix(X.quanti=split$X.quanti,  
                     X.quali=split$X.quali, 
                     rename.level=TRUE, 
                     graph=FALSE, 
                     ndim=25)

## Inspect principal components
res.pcamix$eig

,Eigenvalue,Proportion,Cumulative
dim 1,4.5098513842,20.49932447,20.49932
dim 2,2.8024012638,12.73818756,33.23751
dim 3,1.8247464408,8.29430200,41.53181
dim 4,1.1490819680,5.22309985,46.75491
dim 5,1.0495900326,4.77086378,51.52578
dim 6,1.0144020899,4.61091859,56.13670
dim 7,0.9993340111,4.54242732,60.67912
dim 8,0.9842737776,4.47397172,65.15310
dim 9,0.9039827483,4.10901249,69.26211
dim 10,0.8465510646,3.84795938,73.11007


We see that the results from `FactoMineR` and `PCAmixdata` are identical.

A little background: an **eigenvalue > 1** indicates that the principal component (PCs) accounts for **more** variance than accounted by one of the original variables in **standardized** data (**N.B. This holds true only when the data are standardized.**). This is commonly used as a cutoff point for which PCs are retained. 

Therefore, interestingly, only the **first four** PCs account for more variance than each of the original variables, and together they account for only 46.7% of the total variance in the data set. This suggests that patterns between the variables are likely non-linear and complex.

## 3.3 `prince`

Unlike the two packages above, `prince` is implemented in Python.

In [12]:
## Import libraries
import prince
import fbpca

## Instantiate FAMD object
famd = prince.FAMD(
     n_components=25,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',   ## Can be "auto", 'sklearn', 'fbpca'
     random_state=42)

## Fit FAMD object to data (excluding target varible)
famd = famd.fit(df.drop('Churn', axis=1))

## Inspect principal dimensions
famd.explained_inertia_            

[0.5374654125558428,
 0.08801867828903982,
 0.0572851244368975,
 0.03937333023240256,
 0.03127877295810106,
 0.027912251575853336,
 0.024703045349880923,
 0.020807301704142356,
 0.018937230974685852,
 0.01800539646894815,
 0.01656022774266522,
 0.015976757070266686,
 0.014945093457876379,
 0.013999456118270703,
 0.013763376399511574,
 0.013589916692549978,
 0.012208295034160313,
 0.011979368941411956,
 0.011339889947465936,
 0.007002655988383335,
 0.0048478677362699864,
 5.485296596457979e-07,
 1.795713232509243e-09,
 5.3660658752776136e-33,
 5.3660658752776136e-33]

Surprisingly, the results here differ greatly from the ones above. In my preliminary readings, I understand that "explained inertia" is synonymous with "explained variance", so this seems to be an unlikely cause of the discrepancy. I will keep digging, but as you will see in later parts of this series, FAMD performed using `prince` does reach nearly identical conclusions as the two R packages.